In [ ]:
!pip install torch transformers x-transformers betacal captum 

In [ ]:
# # Basic imports
import numpy as np  # For numerical computations and array manipulations
import pandas as pd  # For loading and handling time-series and static data
import sys
import importlib
import os
import time
from tqdm import tqdm
# PyTorch imports
import torch  # Core PyTorch library
import torch.nn as nn  # Neural network layers and loss functions
import torch.optim as optim  # Optimization algorithms
from torch.utils.data import Dataset, DataLoader  # Datasets and DataLoaders for batching
from torch.nn import Transformer, TransformerEncoderLayer  # Transformer modules
import torch.nn.functional as F
from captum.attr import IntegratedGradients
import re
# #Tranformers import
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import accuracy_score, precision_recall_curve, auc, roc_auc_score


module_path = '/home/workspace/files/MilanK/Model1/final_models/code'
# Add the module's directory to the system path if it's not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
    

from generating_datasets_for_torch import *
from load_static_data import *
from PatientDataset import *
from generate_labels_dataframe_with_dataloader import *
from load_train_test_split import *
from model import *
from load_patient_list import *
from forward_loop import *
from fit import *
from validate import *

In [ ]:
train,val,test = load_train_test_data(
    train_filename = 'train_patient_list_orig.txt',                                   
    val_filename = 'val_patient_list_orig.txt',
    test_filename = 'test_patient_list.txt'
)

test_dataset = PatientDataset(patient_list = test, min_window_min=15, step_min=15,max_window_min=15,
                             prediction_window_length=15)

batch_size = 8
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    #prefetch_factor=1,
    #persistent_workers=True
)


In [ ]:
df = pd.read_csv('/home/workspace/files/MilanK/Model1/final_models/resp_results.csv')
df

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from captum.attr import IntegratedGradients

# ==============================
# 1. YOUR DATA PREPARATION CODE
# ==============================
# ... e.g., define test_dataset, test_loader, model, etc. ...

threshold = 0.443290

# Example dynamic columns in correct order
final_dynamic_columns = [
    'nacooMin', 'nCoo', 'ntempEs', 'ntempSk', 'ntempCo', 'nTemp',
    'nawRr', 'nrespRate', 'nartMean', 'nartDia', 'nartSys',
    'nnbpMean', 'nnbpDia', 'nnbpSys', 'nPleth', 'necgRate',
    'previous_resp_deteriorations','previous_cardiovascular_deteriorations'
]

# Example scalar columns
scalar_column_names = [
    'age_months',
    'age_months_missing',
    'gender',
    'gender_missing',
    'weight_kg',
    'weight_missing',
    'ventilation_status',
    'ventilation_status_missing',
    'pim3',
    'Destination Care Area_missing',
    'Destination Care Area_HDU (step-up / step-down unit)',
    'Destination Care Area_ICU',
    'Destination Care Area_NICU',
    'Destination Care Area_PICU',
    'Destination Care Area_Ward',
    'Destination Care Area_Other',
    'vasoactive_agent_used',
    'vasoactive_agent_used_misisng',
    'Adrenaline',
    'Dobutamine',
    'Dopamine',
    'Milrinone',
    'Noradrenaline',
    'Prostaglandin',
    'Vasopressin',
    'total_vasoactive_agents',
    'Neurological',
    'Cardiac',
    'Respiratory',
    'Multi-system',
    'Genetic / Syndrome',
    'Metabolic / Endocrine',
    'Haem / Onc',
    'Other',
    'Renal',
    'None',
    'preexisting_conditions_missing',
    'total_pre_existing_condition',
    'day_night',
    'primary_diagnosis_missing'
]

# Feature group lists (optional)
age_cols = ["age_months", "age_months_missing"]
sex_cols = ["gender", "gender_missing"]
weight_cols = ["weight_kg", "weight_missing"]
ventilation_cols = ["ventilation_status", "ventilation_status_missing"]
dest_area_cols = [
    "Destination Care Area_missing",
    "Destination Care Area_HDU (step-up / step-down unit)",
    "Destination Care Area_ICU",
    "Destination Care Area_NICU",
    "Destination Care Area_PICU",
    "Destination Care Area_Ward",
    "Destination Care Area_Other"
]
medical_history_columns = [
    "Neurological", "Cardiac", "Respiratory", "Multi-system", "Genetic / Syndrome",
    "Metabolic / Endocrine", "Haem / Onc", "Other", "Renal", "None",
    "preexisting_conditions_missing", "total_pre_existing_condition"
]
vasoactive_columns = [
    "vasoactive_agent_used",
    "vasoactive_agent_used_misisng",
    "Adrenaline",
    "Dobutamine",
    "Dopamine",
    "Milrinone",
    "Noradrenaline",
    "Prostaglandin",
    "Vasopressin",
    "total_vasoactive_agents"
]

# ===========================
# 2. LOAD YOUR MODEL & SETUP
# ===========================
model = load_model_for_eval()  # your custom function
device = torch.device('cpu')   # or "cuda" if GPU is available
model.to(device)
model.eval()

ig = IntegratedGradients(model)

# Create lists for results
results_directional = []  # For raw signed attributions
results_magnitude = []    # For absolute attributions

patient_counter = 0

# ======================================
# 3. COMPUTE INTEGRATED GRADIENTS PER BATCH
# ======================================
for batch_idx, batch in enumerate(test_loader):
    (
        test_dynamic,
        test_missing,
        _,
        _,
        test_scalar,
        test_list,
        test_label,
        *_  # ignore any extra items from the dataset
    ) = batch

    # Move inputs to device
    test_dynamic = test_dynamic.to(device)
    test_missing = test_missing.to(device)
    test_scalar = test_scalar.to(device)
    test_list   = test_list.to(device)
    test_label  = test_label.to(device)

    # Model predictions
    with torch.no_grad():
        logits = model(test_dynamic, test_missing, test_scalar, test_list)
        probs = torch.sigmoid(logits)
        preds = (probs > threshold).long()

    # Prepare zero baselines
    baseline_dynamic = torch.zeros_like(test_dynamic)
    baseline_missing = torch.zeros_like(test_missing)
    baseline_scalar  = torch.zeros_like(test_scalar)
    baseline_list    = torch.zeros_like(test_list)

    # Calculate attributions
    attributions, delta = ig.attribute(
        inputs     = (test_dynamic, test_missing, test_scalar, test_list),
        baselines  = (baseline_dynamic, baseline_missing, baseline_scalar, baseline_list),
        target     = 0,  # or whichever class index is relevant
        return_convergence_delta=True
    )

    # Unpack attributions
    dynamic_attr, mask_attr, scalar_attr, embedding_attr = attributions

    # -------------------
    # 3A. DIRECTIONAL (SIGNED)
    # -------------------
    # Sum across time dimension but preserve sign
    dynamic_importance_raw = dynamic_attr.sum(dim=1)  
    mask_importance_raw    = mask_attr.sum(dim=1)     
    dynamic_importance_raw = dynamic_importance_raw + mask_importance_raw

    # Scalars are already single-step
    scalar_importance_raw = scalar_attr

    # Embedding: sum across embedding dimension
    embedding_importance_raw = embedding_attr.sum(dim=1)

    # -------------------
    # 3B. MAGNITUDE (ABSOLUTE)
    # -------------------
    dynamic_importance_mag = dynamic_attr.abs().sum(dim=1)
    mask_importance_mag    = mask_attr.abs().sum(dim=1)
    dynamic_importance_mag = dynamic_importance_mag + mask_importance_mag

    scalar_importance_mag  = scalar_attr.abs()
    embedding_importance_mag = embedding_attr.abs().sum(dim=1)

    # Move back to CPU numpy
    probs_np  = probs.detach().cpu().numpy()
    preds_np  = preds.detach().cpu().numpy()
    labels_np = test_label.detach().cpu().numpy()

    dyn_raw_np       = dynamic_importance_raw.detach().cpu().numpy()
    scalar_raw_np    = scalar_importance_raw.detach().cpu().numpy()
    embed_raw_np     = embedding_importance_raw.detach().cpu().numpy()

    dyn_mag_np       = dynamic_importance_mag.detach().cpu().numpy()
    scalar_mag_np    = scalar_importance_mag.detach().cpu().numpy()
    embed_mag_np     = embedding_importance_mag.detach().cpu().numpy()

    # -------------------------------
    # 3C. BUILD PER-PATIENT DICTIONARIES
    # -------------------------------
    batch_size = dyn_raw_np.shape[0]
    for i in range(batch_size):
        # DIRECTIONAL ROW
        row_dict_dir = {}
        row_dict_dir["patient_id"] = patient_counter

        # Dynamic features
        for j, val in enumerate(dyn_raw_np[i]):
            if j < len(final_dynamic_columns):
                row_dict_dir[final_dynamic_columns[j]] = float(val)
            else:
                row_dict_dir[f"dynamic_{j}"] = float(val)

        # Scalar features
        for j, val in enumerate(scalar_raw_np[i]):
            row_dict_dir[scalar_column_names[j]] = float(val)

        # Embedding
        row_dict_dir["diagnosis_embedding"] = float(embed_raw_np[i])

        # Predictions
        row_dict_dir["y_prob"]        = float(probs_np[i])
        row_dict_dir["y_pred"]        = int(preds_np[i])
        row_dict_dir["y_true_resp"] = int(labels_np[i][1])  # or however your label is stored

        results_directional.append(row_dict_dir)

        # MAGNITUDE ROW
        row_dict_mag = {}
        row_dict_mag["patient_id"] = patient_counter

        # Dynamic features
        for j, val in enumerate(dyn_mag_np[i]):
            if j < len(final_dynamic_columns):
                row_dict_mag[final_dynamic_columns[j]] = float(val)
            else:
                row_dict_mag[f"dynamic_{j}"] = float(val)

        # Scalar features
        for j, val in enumerate(scalar_mag_np[i]):
            row_dict_mag[scalar_column_names[j]] = float(val)

        # Embedding
        row_dict_mag["diagnosis_embedding"] = float(embed_mag_np[i])

        # Predictions
        row_dict_mag["y_prob"]        = float(probs_np[i])
        row_dict_mag["y_pred"]        = int(preds_np[i])
        row_dict_mag["y_true_resp"] = int(labels_np[i][0])

        results_magnitude.append(row_dict_mag)

        patient_counter += 1

# ======================
# 4. CREATE DATAFRAMES
# ======================
df_directional = pd.DataFrame(results_directional)
df_magnitude   = pd.DataFrame(results_magnitude)

# -------------------------------------------------------
# 5. GROUP FEATURE IMPORTANCE (OPTIONAL)
# -------------------------------------------------------
def aggregate_feature_groups(df):
    """
    Aggregate columns into broader feature groups (e.g. sum up age columns).
    Adjust as needed for your use-case. 
    """
    df["age"]                  = df[age_cols].sum(axis=1)
    df["sex"]                  = df[sex_cols].sum(axis=1)
    df["weight"]               = df[weight_cols].sum(axis=1)
    df["ventilation_support"]  = df[ventilation_cols].sum(axis=1)
    df["destination_care_area"] = df[dest_area_cols].sum(axis=1)
    df["medical_history"]      = df[medical_history_columns].sum(axis=1)
    df["vasoactive_agents"]    = df[vasoactive_columns].sum(axis=1)

    # Drop the old columns that were grouped
    drop_cols = (
        age_cols + sex_cols + weight_cols + ventilation_cols +
        dest_area_cols + medical_history_columns + vasoactive_columns +
        ["primary_diagnosis_missing"]  # if you want to drop it
    )
    # Only drop columns that exist
    drop_cols = [c for c in drop_cols if c in df.columns]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')

    return df

df_directional = aggregate_feature_groups(df_directional)
df_magnitude   = aggregate_feature_groups(df_magnitude)

# -----------------------------------
# 6. NORMALIZATION UTILITIES
# -----------------------------------
prediction_cols = ['y_prob', 'y_pred', 'y_true_resp']

def normalize_magnitude_rows(df, prediction_cols):
    """
    Normalizes *absolute* attributions row-wise so each patient's
    features sum to 1.
    """
    feature_cols = [c for c in df.columns if c not in prediction_cols + ["patient_id"]]
    row_sums = df[feature_cols].sum(axis=1)
    df_norm_features = df[feature_cols].div(row_sums, axis=0).fillna(0)
    df_norm = pd.concat([df[['patient_id']], df_norm_features, df[prediction_cols]], axis=1)
    return df_norm

def normalize_signed_attributions(df, prediction_cols):
    """
    Normalizes *signed* attributions row-wise by the sum of absolute values,
    so positive values stay positive, negative stay negative,
    and sum of absolute values becomes 1 for each row.
    """
    feature_cols = [c for c in df.columns if c not in prediction_cols + ["patient_id"]]
    row_abs_sum = df[feature_cols].abs().sum(axis=1)
    df_norm_features = df[feature_cols].div(row_abs_sum, axis=0).fillna(0)
    df_norm = pd.concat([df[['patient_id']], df_norm_features, df[prediction_cols]], axis=1)
    return df_norm

# Make the normalized DataFrames
df_magnitude_norm   = normalize_magnitude_rows(df_magnitude.copy(), prediction_cols)
df_directional_norm = normalize_signed_attributions(df_directional.copy(), prediction_cols)

# ===========================
# 7. SAVE OR USE THE DATAFRAMES
# ===========================
# e.g.:
# df_directional.to_csv("integrated_gradients_directional.csv", index=False)
# df_magnitude.to_csv("integrated_gradients_magnitude.csv", index=False)
df_magnitude_norm.to_csv("/home/workspace/files/MilanK/Model1/final_models/final_resp_models/combined_model_simpler_demographics/integrated_gradients_magnitude_norm.csv", index=False)
df_directional_norm.to_csv("/home/workspace/files/MilanK/Model1/final_models/final_resp_models/combined_model_simpler_demographics/integrated_gradients_directional_norm.csv", index=False)

print("Directional (raw) shape:", df_directional.shape)
print("Magnitude (raw) shape:", df_magnitude.shape)
print("Directional normalized shape:", df_directional_norm.shape)
print("Magnitude normalized shape:", df_magnitude_norm.shape)
